# Phase 4 NLP ML Project

- What trends are we seeing. What do people like

![Image Description](Image/Twitter1.png)

# Tensor Flow Details

In [ ]:
conda activate TFgpu

# Load Python Packages & DF

In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer, sent_tokenize
import string
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from wordcloud import WordCloud
from afinn import Afinn
from nltk.corpus import sentiwordnet as swn
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import unicodedata
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier

In [2]:
# Read the dataset
df = pd.read_csv('judge-1377884607_tweet_product_company.csv', encoding='latin1')

# Remove tweets with unknown sentiment
df = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"]

In [5]:
class TextPreprocessor:
    def __init__(self, custom_stopwords=None):
        self.custom_stopwords = custom_stopwords

    def clean_text(self, text):
        if pd.isnull(text):
            return ''
        else:
            text = text.lower()
            text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
            text = re.sub(r'@\w+', 'USER', text)
            text = re.sub(r'http\S+|www\S+', 'URL', text)
            text = re.sub(r'[^\w\s]', '', text)
            tokens = nltk.word_tokenize(text)
            stop_words = set(stopwords.words('english'))
            if self.custom_stopwords:
                stop_words.update(self.custom_stopwords)
            tokens = [token for token in tokens if token not in stop_words]
            lemmatizer = WordNetLemmatizer()
            tokens = [lemmatizer.lemmatize(token) for token in tokens]
            return ' '.join(tokens)

In [ ]:
# class TextPreprocessor(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.stop_words = set(stopwords.words('english'))
    
#     def preprocess_text(self, text):
#         if isinstance(text, str):  # Check if text is a non-null string
#             # Remove punctuation and convert to lowercase
#             text = ''.join([char.lower() for char in text if char.isalnum() or char.isspace()])
#             # Remove stop words
#             text = ' '.join([word for word in text.split() if word not in self.stop_words])
#         else:
#             text = ''  # Replace NaN values with an empty string
#         return text
    
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         return [self.preprocess_text(text) for text in X]

In [6]:
# Load and preprocess the data
X = df['tweet_text']
y = df['is_there_an_emotion_directed_at_a_brand_or_product']

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', DummyClassifier(strategy='most_frequent'))  # Dummy Classifier
])

# Train the model and make predictions
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' '<__main__.TextPreprocessor object at 0x00000176792DCA00>' (type <class '__main__.TextPreprocessor'>) doesn't

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', MultinomialNB())  # Naive Bayes classifier
])

# Define the hyperparameters to search
hyperparameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # N-gram range for TF-IDF
    'classifier__alpha': [0.1, 1.0, 10.0]  # Smoothing parameter for Naive Bayes
}

# Perform grid search
grid_search = GridSearchCV(pipeline, hyperparameters, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model and its accuracy
best_model = grid_search.best_estimator_
best_accuracy = grid_search.best_score_
print(f'Best Accuracy: {best_accuracy}')

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

# Whole Model

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', XGBClassifier())  # XGBoost classifier
])

# Train the model and make predictions
pipeline.fit(X_train, y_train_encoded)
y_pred_encoded = pipeline.predict(X_test)

# Decode the predictions
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
#Tweet tokenizer NLTK 

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
    
    def preprocess_text(self, text):
        if isinstance(text, str):  # Check if text is a non-null string
            # Remove punctuation and convert to lowercase
            text = ''.join([char.lower() for char in text if char.isalnum() or char.isspace()])
            # Remove stop words
            text = ' '.join([word for word in text.split() if word not in self.stop_words])
        else:
            text = ''  # Replace NaN values with an empty string
        return text
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [self.preprocess_text(text) for text in X]

In [ ]:
# Read the dataset
df = pd.read_csv('judge-1377884607_tweet_product_company.csv', encoding='latin1')

In [ ]:
# Remove tweets with unknown sentiment
df = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"]

In [ ]:
# Load and preprocess the data
X = df['tweet_text']
y = df['is_there_an_emotion_directed_at_a_brand_or_product']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', DummyClassifier(strategy='most_frequent'))  # Dummy Classifier
])

# Train the model and make predictions
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', LinearSVC())  # Linear Support Vector Classifier (SVC)
])

# Train the model and make predictions
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', MultinomialNB())  # Naive Bayes classifier
])

# Train the model and make predictions
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', XGBClassifier())  # XGBoost classifier
])

# Train the model and make predictions
pipeline.fit(X_train, y_train_encoded)
y_pred_encoded = pipeline.predict(X_test)

# Decode the predictions
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# XG Boost Dec Tree

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', XGBClassifier())  # XGBoost classifier
])

# Define the hyperparameters to search
hyperparameters = {
    'classifier__n_estimators': [100, 200, 300],  # Number of trees in the forest
    'classifier__learning_rate': [0.1, 0.01, 0.001],  # Learning rate for each tree
    'classifier__max_depth': [3, 5, 7]  # Maximum depth of each tree
}

# Perform grid search
grid_search = GridSearchCV(pipeline, hyperparameters, cv=5)
grid_search.fit(X_train, y_train_encoded)

# Get the best model and its accuracy
best_model = grid_search.best_estimator_
best_accuracy = grid_search.best_score_
print(f'Best Accuracy: {best_accuracy}')

# Make predictions using the best model
y_pred_encoded = best_model.predict(X_test)

# Decode the predictions
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')


# Grid Search NB

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', MultinomialNB())  # Naive Bayes classifier
])

# Define the hyperparameters to search
hyperparameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # N-gram range for TF-IDF
    'classifier__alpha': [0.1, 1.0, 10.0]  # Smoothing parameter for Naive Bayes
}

# Perform grid search
grid_search = GridSearchCV(pipeline, hyperparameters, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model and its accuracy
best_model = grid_search.best_estimator_
best_accuracy = grid_search.best_score_
print(f'Best Accuracy: {best_accuracy}')

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

In [ ]:
# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
    ('classifier', RandomForestClassifier())  # Random Forest classifier
])

# Train the model and make predictions
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.base import TransformerMixin

class TextPreprocessor(TransformerMixin):
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
    
    def preprocess_text(self, text):
        if isinstance(text, str):  # Check if text is a non-null string
            # Remove punctuation and convert to lowercase
            text = ''.join([char.lower() for char in text if char.isalnum() or char.isspace()])
            # Remove stop words
            text = ' '.join([word for word in text.split() if word not in self.stop_words])
        else:
            text = ''  # Replace NaN values with an empty string
        return text
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [self.preprocess_text(text) for text in X]
    
    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y)
        return self.transform(X)

# Load and preprocess the data
df = pd.read_csv('judge-1377884607_tweet_product_company.csv', encoding='latin1')
X = df['tweet_text']
y = df['is_there_an_emotion_directed_at_a_brand_or_product']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', TextPreprocessor()),  # Custom text preprocessing
    ('tfidf', TfidfVectorizer()),  # TF-IDF feature extraction
])

# Preprocess the text data
X_train_processed = pipeline.named_steps['preprocessor'].transform(X_train)
X_test_processed = pipeline.named_steps['preprocessor'].transform(X_test)

# Convert the processed text data to a list of strings
X_train_processed = [str(text) for text in X_train_processed]
X_test_processed = [str(text) for text in X_test_processed]

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_processed)

X_train_tokenized = tokenizer.texts_to_sequences(X_train_processed)
X_test_tokenized = tokenizer.texts_to_sequences(X_test_processed)

# Pad sequences
max_sequence_length = max(max(len(x) for x in X_train_tokenized), max(len(x) for x in X_test_tokenized))
X_train_padded = pad_sequences(X_train_tokenized, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_tokenized, maxlen=max_sequence_length)

# Create the deep learning model
embedding_dim = 100
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert target variable to binary values
y_train_binary = (y_train == 'positive').astype(int)
y_test_binary = (y_test == 'positive').astype(int)

# Train the model
model.fit(X_train_padded, y_train_binary, validation_data=(X_test_padded, y_test_binary), epochs=10, batch_size=32)

# Predict probabilities
y_pred_prob = model.predict(X_test_padded)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_binary, y_pred)
print(f'Accuracy: {accuracy}')


In [ ]:
# Clean and preprocess the text
def clean_text(text, custom_stopwords=None):
    if pd.isnull(text):
        return ''
    else:
        # Convert to lowercase
        text = text.lower()

        # Remove non-ASCII characters
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

        # Remove mentions and URLs
        text = re.sub(r'@\w+', 'USER', text)
        text = re.sub(r'http\S+|www\S+', 'URL', text)

        # Remove special characters and symbols
        text = re.sub(r'[^\w\s]', '', text)

        # Tokenization
        tokens = nltk.word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        if custom_stopwords:
            stop_words.update(custom_stopwords)
        tokens = [token for token in tokens if token not in stop_words]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        return ' '.join(tokens)
